# Stellar luminosity:
## Polynomial Model for Regression

In this notebook, Polynomial regression will be implemented to model the mass-luminosity relationship of stars. 

### Objective

Capture nonlinear and interaction effects using polynomial feature engineering and an explicit bias term:

L_hat = X @ w + b.

### Dataset and Notation

We will use the following notation for the notebook

- **M:** stellar mass (in units of solar mass, M⊙)
- **L:** stellar luminosity (in units of solar luminosity, L⊙)
- **T:** effective stellar temperature (Kelvin, K)

And the dataset for this first part is:

M = [0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4]
L = [0.15, 0.35, 1.00, 2.30, 4.10, 7.00, 11.2, 17.5, 25.0, 35.0]
T = [3800, 4400, 5800, 6400, 6900, 7400, 7900, 8300, 8800, 9200]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
#Set variables in arrays (one feature)
M = np.array([0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4])
L = np.array([0.15, 0.35, 1.00, 2.30, 4.10, 7.00, 11.2, 17.5, 25.0, 35.0])
T = np.array([3800, 4400, 5800, 6400, 6900, 7400, 7900, 8300, 8800, 9200])

N = len(L)  # number of data samples

#### 1. Dataset visualization: M vs L, encoding T

We first visualize the relationship between stellar mass (M) and luminosity (L).
The effective temperature (T) is encoded using color to observe how temperature
correlates with luminosity at different masses.

In [ ]:
plt.figure()
scatter = plt.scatter(M, L, c=T, s=80)
plt.xlabel("Stellar Mass M (M☉)")
plt.ylabel("Luminosity L (L☉)")
plt.title("Luminosity vs Mass (color = Temperature)")
plt.colorbar(scatter, label="Temperature (K)")
plt.show()

Luminosity increases non-linearly with mass. Higher temperatures generally
correspond to higher luminosities, suggesting that temperature should be included
as an explanatory feature.

#### 2. Feature engineering: Design matrix X

Construct the design matrix using the following features (do not include a constant column of ones):

$$X = [ M, T, M^2, M*T ]$$

In [ ]:
def build_X(M, T, model="M3"):
    if model == "M1":
        return np.column_stack([M, T])
    elif model == "M2":
        return np.column_stack([M, T, M**2])
    elif model == "M3":
        return np.column_stack([M, T, M**2, M*T])

#### 3. Loss function and gradients (vectorized)

##### Model

In this notebook, a **polynomial regression model** is used.


$$\hat{L} = Xw + b$$


#### MSE

And in this notebook, also the **Mean Squared Error** is used.

$$J(w,b) = \frac{1}{N} \sum_{i=1}^{N} (\hat{L}_i - L_i)^2$$

In [ ]:
def predict(X, w, b):
    return X @ w + b


def mse(X, L, w, b):
    error = predict(X, w, b) - L
    return np.mean(error**2)


def gradients(X, L, w, b):
    error = predict(X, w, b) - L
    dw = (2 / N) * X.T @ error
    db = (2 / N) * np.sum(error)
    return dw, db

#### 4. Gradient Descent + Convergence


In [ ]:
def gradient_descent(X, L, lr=0.01, iterations=5000):
    w = np.zeros(X.shape[1])
    b = 0.0
    loss_history = []

    for _ in range(iterations):
        dw, db = gradients(X, L, w, b)
        w -= lr * dw
        b -= lr * db
        loss_history.append(mse(X, L, w, b))

    return w, b, loss_history

In [ ]:
M_norm = (M - np.mean(M)) / np.std(M)
T_norm = (T - np.mean(T)) / np.std(T)
L_norm = (L - np.mean(L)) / np.std(L)


#Convergence plot (example with M3)
X3 = build_X(M_norm, T_norm, "M3")
w3, b3, loss3 = gradient_descent(X3, L_norm)

plt.figure()
plt.plot(loss3)
plt.xlabel("Iterations")
plt.ylabel("MSE")
plt.title("Convergence of Gradient Descent (M3)")
plt.show()

The loss decreases smoothly, indicating stable convergence of gradient descent.

#### 5. Feature selection experiment (MANDATORY)

Models:
- **M1:** [M, T]
- **M2:** [M, T, M²]
- **M3:** [M, T, M², M·T]

In [ ]:
models = ["M1", "M2", "M3"]
results = {}

for model in models:
    X = build_X(M_norm, T_norm, model)
    w, b, loss = gradient_descent(X, L_norm)
    results[model] = (X, w, b, loss[-1])

##### Predicted vs Actual plots

In [ ]:
for model, (X, w, b, final_loss) in results.items():
    L_pred = predict(X, w, b)

    plt.figure()
    plt.scatter(L_norm, L_pred)
    plt.plot([0, 5], [0, 5], "--")
    plt.xlabel("Actual Luminosity")
    plt.ylabel("Predicted Luminosity")
    plt.title(f"{model} – Predicted vs Actual (Loss={final_loss:.3f})")
    plt.show()

    print(f"{model}:")
    print("w =", w)
    print("b =", b)
    print("Final loss =", final_loss)
    print()

Model M3 achieves the lowest loss, indicating that both non-linear and interaction
terms improve predictive performance.

#### 6. Cost vs interaction coefficient w_MT (MANDATORY)

We vary the interaction coefficient $w_{MT}$ while keeping all other parameters fixed.

In [ ]:
w_fixed = w3.copy()
w_mt_values = np.linspace(w3[-1] - 1e-6, w3[-1] + 1e-6, 100)
costs = []

for val in w_mt_values:
    w_test = w_fixed.copy()
    w_test[-1] = val
    costs.append(mse(X3, L, w_test, b3))

In [ ]:
plt.figure()
plt.plot(w_mt_values, costs)
plt.xlabel("Interaction coefficient w_MT")
plt.ylabel("Cost (MSE)")
plt.title("Cost vs Interaction Term")
plt.show()

The cost curve exhibits a clear minimum, indicating that the interaction term
between mass and temperature plays a significant role in explaining luminosity.

#### 7. Inference demo (MANDATORY)

New star:
- M = 1.3
- T = 6600

In [ ]:
M_new = np.array([1.3])
T_new = np.array([6600])

X_new = build_X(M_new, T_new, "M3")
L_new_pred = predict(X_new, w3, b3)

L_new_pred

The predicted luminosity lies between nearby observed values, making it physically
reasonable given the star's mass and temperature.

### Final

Including polynomial and interaction features significantly improves model accuracy.
This demonstrates that stellar luminosity depends on both non-linear mass effects
and mass–temperature coupling.